**AUTOCORRELATION PLOTS AND HOW TO INTERPRET THEM**

Interpreting and reading an autocorrelation plot:

The autocorrelation plot shows the correlation of the time series data with itself at different lags.
The x-axis represents the lag, which is the time shift.
The y-axis represents the autocorrelation coefficient, which measures the strength and direction of the correlation.
The shaded region represents the confidence intervals, typically at 95% confidence level. Points outside this region are considered statistically significant.
A positive autocorrelation indicates a positive relationship between the current observation and the observation at the lagged time point.
A negative autocorrelation indicates a negative relationship between the current observation and the observation at the lagged time point.
Autocorrelation values closer to 1 or -1 indicate a stronger correlation, while values closer to 0 indicate weaker correlation.

**WHY NOT TO USE TRAIN TEST SPLIT WITH TIME SERIES**

Here's a simple explanation of why it's not a good idea to shuffle data when working with time series:

Imagine that you have a bunch of data points that represent things that happened over time. Each data point has a time stamp, like a date, that tells you when it happened.

Now, imagine that you mix up all of the data points and put them in a different order. This is like shuffling a deck of cards. It might seem like a good idea because it helps you check if your model is working well, but it's actually not a good idea because the order of the data points is important.

The data points are like a story, and the order they happened in is important to understanding the story. If you shuffle the data points, it's like telling the story out of order, and it doesn't make sense anymore. It's hard for your model to learn from the data if the data doesn't make sense.

So, when you're working with time series data, it's important to keep the data points in the order that they happened. This way, your model can learn from the data and make good predictions.

In [ ]:
# load packages
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, precision_score, f1_score, recall_score
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV

In [ ]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load lagged dataset
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/lagged_curtailment_target_features.csv', sep = ';', index_col=0)

In [ ]:
#converting index from object type to datetime type
df.index = pd.to_datetime(df.index)

In [ ]:
df.columns

Index(['redispatch', 'level', 'wind_speed_m/s', 'wind_speed_m/s_lag1',
       'wind_speed_m/s_lag2', 'wind_speed_m/s_lag3', 'wind_speed_m/s_lag4',
       'wind_speed_m/s_lag5', 'wind_direction_degrees',
       'wind_direction_degrees_lag1', 'wind_direction_degrees_lag2',
       'wind_direction_degrees_lag3', 'wind_direction_degrees_lag4',
       'wind_direction_degrees_lag5', 'radiation_global_J/m2',
       'radiation_global_J/m2_lag1', 'radiation_global_J/m2_lag2',
       'radiation_global_J/m2_lag3', 'radiation_global_J/m2_lag4',
       'radiation_global_J/m2_lag5', 'air_temperature_K',
       'air_temperature_K_lag1', 'air_temperature_K_lag2',
       'air_temperature_K_lag3', 'air_temperature_K_lag4',
       'air_temperature_K_lag5', 'humidity_percent', 'humidity_percent_lag1',
       'humidity_percent_lag2', 'humidity_percent_lag3',
       'humidity_percent_lag4', 'humidity_percent_lag5', 'wind_gust_max_m/s',
       'wind_gust_max_m/s_lag1', 'wind_gust_max_m/s_lag2',
       'wind_g

In [ ]:
# Identify significant lag values (where autocorrelation is outside the confidence intervals)
significant_lags = [48, 96]

df_lagged = df.copy()

# Create lagged features
for lag in significant_lags:
    df_lagged[f'redispatch_lag_{lag}'] = df_lagged['redispatch'].shift(lag)
    df_lagged[f'level_lag_{lag}'] = df_lagged['level'].shift(lag)

# Drop rows with NaN values resulting from the shifting
df_lagged.dropna(inplace=True)


In [ ]:
# get desired df size
start_date = '2022-01-01'
end_date = '2023-06-30'
df_lagged = df_lagged.loc[start_date:end_date]

TESTING XGBOOST WITHOUT SMOTE

but handling class imbalance with scale_pos_weight = ratio

Average Train F1 Score: 0.5314998790499398
Average Train Precision: 0.5898975814577239
Average Test F1 Score: 0.16610137820824686
Average Test Precision: 0.22274850608643465


2. ratio = 5

Average Train F1 Score: 0.47032209963494065
Average Train Precision: 0.6848833081255093
Average Test F1 Score: 0.13780767453150464
Average Test Precision: 0.25665706143264116

3. ratio = 4

Average Train F1 Score: 0.4364925588864709
Average Train Precision: 0.7113404944095644
Average Test F1 Score: 0.1262201619767872
Average Test Precision: 0.26329427123769217

4. ratio =6; threshold = 0.45

Average Train F1 Score: 0.49843291517629684
Average Train Precision: 0.653480149643088
Average Train Recall: 0.44467934155177663

 Average Test F1 Score: 0.1413042411667109
Average Test Precision: 0.25175147014687826
Average Test Recall: 0.12706465765116165

 Average Threshold F1 Score: 0.1659465367087471
Average Threshold Precision: 0.25264179317835117
Average Threshold Recall: 0.16769221937182072

5. ratio = 10; threshold = 0.6; n_splits = 10

Average Train F1 Score: 0.5673634598155803
Average Train Precision: 0.6788178041013995
Average Train Recall: 0.5649233506536049

 Average Test F1 Score: 0.15600429057526033
Average Test Precision: 0.32111401008649665
Average Test Recall: 0.1792308401831178

 Average Threshold F1 Score: 0.12516613010671776
Average Threshold Precision: 0.4552949064158541
Average Threshold Recall: 0.12271878954130425

6. same as above and for all years

Average Train F1 Score: 0.6271271587571896
Average Train Precision: 0.719607310683743
Average Train Recall: 0.6314889332056188

 Average Test F1 Score: 0.15124855685122585
Average Test Precision: 0.41402793661481174
Average Test Recall: 0.18656238738548198

 Average Threshold F1 Score: 0.12338705210525355
Average Threshold Precision: 0.5715504495040475
Average Threshold Recall: 0.1292740934462511

In [ ]:
# Define features and target
features = df_lagged.columns

target = 'redispatch'

X = df_lagged[features].drop(['redispatch', 'level'], axis=1)
y = df_lagged[target]

# Define hyperparameters
params = {
    'max_depth': 7,
    'min_child_weight': 10,
    'gamma': 0.2,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'booster': 'gbtree',
    'reg_alpha': 4,
    'reg_lambda': 4,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'random_state': 42,
    'verbosity': 0,
    #'early_stopping_rounds': 20,
    'scale_pos_weight': 1
}

# Define the number of splits for time series cross-validation
n_splits = 10  # You can adjust this value as needed
gap = 48  # 12 hour difference between train and test sets

# Initialize lists to store evaluation metrics for each fold
train_f1_scores = []
train_precision_scores = []
train_recall_scores = []
test_f1_scores = []
test_precision_scores = []
test_recall_scores = []
thresh_f1_scores = []
thresh_precision_scores = []
thresh_recall_scores = []

# Define the time series splitter
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

# Define threshold for precision calculation
threshold = 0.7

# Instantiate SMOTE
smote = SMOTE(k_neighbors=1, random_state=42)

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
    print(f"Training on fold {fold}/{n_splits}")
    # Get the data for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Apply SMOTE to balance the classes
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Initialize and train the XGBoost classifier with early stopping
    model = XGBClassifier(**params)

    #eval_set = [(X_train_resampled, y_train_resampled), (X_test, y_test)]  # Evaluation sets for early stopping
    model.fit(X_train_resampled, y_train_resampled, verbose=0)#, eval_set=eval_set)

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate the model using precision, recall, and F1 score
    train_f1 = f1_score(y_train, y_train_pred, zero_division=1)
    train_precision = precision_score(y_train, y_train_pred, zero_division=1)
    train_recall = recall_score(y_train, y_train_pred, zero_division=1)
    test_f1 = f1_score(y_test, y_test_pred, zero_division=1)
    test_precision = precision_score(y_test, y_test_pred, zero_division=1)
    test_recall = recall_score(y_test, y_test_pred, zero_division=1)

    # Compute precision, recall, and F1 score using the specified threshold
    y_test_proba = model.predict_proba(X_test)[:, 1]
    y_test_pred_threshold = (y_test_proba > threshold).astype(int)
    fold_precision = precision_score(y_test, y_test_pred_threshold, zero_division=1)
    fold_recall = recall_score(y_test, y_test_pred_threshold, zero_division=1)
    fold_f1 = f1_score(y_test, y_test_pred_threshold, zero_division=1)

    cm = confusion_matrix(y_test, y_test_pred_threshold)
    # Print the confusion matrix
    print(f"Confusion Matrix (Fold {fold}):")
    print(cm)
    print("\n")

    # Store the evaluation metrics for this fold
    train_f1_scores.append(train_f1)
    train_precision_scores.append(train_precision)
    train_recall_scores.append(train_recall)
    test_f1_scores.append(test_f1)
    test_precision_scores.append(test_precision)
    test_recall_scores.append(test_recall)
    thresh_f1_scores.append(fold_f1)
    thresh_precision_scores.append(fold_precision)
    thresh_recall_scores.append(fold_recall)

# Calculate average evaluation metrics across all folds
avg_train_f1 = np.mean(train_f1_scores)
avg_train_precision = np.mean(train_precision_scores)
avg_train_recall = np.mean(train_recall_scores)
avg_test_f1 = np.mean(test_f1_scores)
avg_test_precision = np.mean(test_precision_scores)
avg_test_recall = np.mean(test_recall_scores)
avg_thresh_f1_scores = np.mean(thresh_f1_scores)
avg_thresh_precision_scores = np.mean(thresh_precision_scores)
avg_thresh_recall_scores = np.mean(thresh_recall_scores)

# Print the results
print("Average Train F1 Score:", avg_train_f1)
print("Average Train Precision:", avg_train_precision)
print("Average Train Recall:", avg_train_recall)
print("\n Average Test F1 Score:", avg_test_f1)
print("Average Test Precision:", avg_test_precision)
print("Average Test Recall:", avg_test_recall)
print("\n Average Threshold F1 Score:", avg_thresh_f1_scores)
print("Average Threshold Precision:", avg_thresh_precision_scores)
print("Average Threshold Recall:", avg_thresh_recall_scores)

Training on fold 1/10
Confusion Matrix (Fold 1):
[[4643    0]
 [ 123    0]]


Training on fold 2/10
Confusion Matrix (Fold 2):
[[4211    7]
 [ 548    0]]


Training on fold 3/10
Confusion Matrix (Fold 3):
[[3591  173]
 [ 598  404]]


Training on fold 4/10
Confusion Matrix (Fold 4):
[[3631  205]
 [ 728  202]]


Training on fold 5/10
Confusion Matrix (Fold 5):
[[3064  469]
 [ 841  392]]


Training on fold 6/10
Confusion Matrix (Fold 6):
[[2765  335]
 [1204  462]]


Training on fold 7/10
Confusion Matrix (Fold 7):
[[3985  195]
 [ 548   38]]


Training on fold 8/10
Confusion Matrix (Fold 8):
[[4095  128]
 [ 472   71]]


Training on fold 9/10
Confusion Matrix (Fold 9):
[[4200  165]
 [ 232  169]]


Training on fold 10/10
Confusion Matrix (Fold 10):
[[4405  225]
 [  67   69]]


Average Train F1 Score: 0.7901081619647353
Average Train Precision: 0.7418472727699695
Average Train Recall: 0.8457980695236111

 Average Test F1 Score: 0.3035361744208359
Average Test Precision: 0.3232018962743025
Ave

In [ ]:
# safe XGBoost classifier
joblib.dump(model, '/content/drive/My Drive/ms_wind_curtailment_prediction/xgboost_class.pkl')

['/content/drive/My Drive/ms_wind_curtailment_prediction/xgboost_class.pkl']

In [ ]:
# impute missing values for forecast solar with actual solar
for index, row in df.iterrows():
    if pd.isna(row['forecast_solar_MW']):
        df.at[index, 'forecast_solar_MW'] = row['actual_solar_MW']

# impute other missing values by interpolation
columns_to_interpolate = ["wind_speed_m/s",  "wind_direction_degrees", "humidity_percent", "radiation_global_J/m2", "air_temperature_K", "wind_gust_max_m/s", "wind_direction_gust_max_degrees", "forecast_solar_MW", "total_grid_load_MWh", "residual_load_MWh", "pumped_storage_MWh"]
df[columns_to_interpolate] = df[columns_to_interpolate].interpolate(method='linear', limit_direction='both')
df.drop("actual_solar_MW", axis = 1, inplace = True)

In [ ]:
'''# Assuming df is your DataFrame with a datetime index
# Create a new column for weekday/weekend
df['weekday'] = df.index.weekday < 5

# Create a new column for season based on month
df['season'] = (df.index.month % 12 + 3) // 3

# Perform one-hot encoding for the 'season' column
df = pd.get_dummies(df, columns=['season'], drop_first=True)

# Perform one-hot encoding for the 'weekday' column
df = pd.get_dummies(df, columns=['weekday'], drop_first=True)

# Convert the 'weekday' and 'season' columns to numerical
df['weekday'] = df['weekday_True'].astype(int)
df['season'] = df[['season_2', 'season_3', 'season_4']].idxmax(axis=1).str.extract(r'(\d)').astype(int)

# Drop the intermediate columns created during one-hot encoding
df.drop(columns=['weekday_True', 'season_2', 'season_3', 'season_4'], inplace=True)

# Now df contains the original data with the new weekday and season columns encoded numerically
'''

**TESTING XGBOOST WITHOUT LAGGED FEATURES**

1. no. of splits is 10
2. train data is smoted
3. level is dropped
4. gap = 48, indicating a 12 hour time difference between train and test sets

In [ ]:
# Define features and target
features = ['wind_speed_m/s', 'wind_direction_degrees', 'radiation_global_J/m2',
            'air_temperature_K', 'humidity_percent', 'wind_gust_max_m/s',
            'wind_direction_gust_max_degrees', 'forecast_solar_MW',
            'total_grid_load_MWh', 'residual_load_MWh', 'pumped_storage_MWh']
target = 'redispatch'

X = df[features]
y = df[target]

# Define the number of splits for time series cross-validation
n_splits = 10  # You can adjust this value as needed
gap = 48 #12 hour difference between train and test sets

# Initialize lists to store evaluation metrics for each fold
train_f1_scores = []
train_precision_scores = []
test_f1_scores = []
test_precision_scores = []

# Define the time series splitter
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

# Iterate over each fold
for train_index, test_index in tscv.split(X):
    # Get the data for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Handle class imbalance with SMOTE
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Initialize and train the XGBoost classifier
    model = XGBClassifier()
    model.fit(X_train_resampled, y_train_resampled)

    # Predictions
    y_train_pred = model.predict(X_train_resampled)
    y_test_pred = model.predict(X_test)

    # Evaluate the model using precision and F1 score
    train_f1 = f1_score(y_train_resampled, y_train_pred)
    train_precision = precision_score(y_train_resampled, y_train_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)

    # Store the evaluation metrics for this fold
    train_f1_scores.append(train_f1)
    train_precision_scores.append(train_precision)
    test_f1_scores.append(test_f1)
    test_precision_scores.append(test_precision)

# Calculate average evaluation metrics across all folds
avg_train_f1 = np.mean(train_f1_scores)
avg_train_precision = np.mean(train_precision_scores)
avg_test_f1 = np.mean(test_f1_scores)
avg_test_precision = np.mean(test_precision_scores)

# Print the results
print("Average Train F1 Score:", avg_train_f1)
print("Average Train Precision:", avg_train_precision)
print("Average Test F1 Score:", avg_test_f1)
print("Average Test Precision:", avg_test_precision)

Average Train F1 Score: 0.984944808880309
Average Train Precision: 0.982622027331205
Average Test F1 Score: 0.06338456555898733
Average Test Precision: 0.08839974275175339


**TESTING XGBOOST WITH LAGGED FEATURES**

1. test 1 with 2 new cols
  - added redispatch lag of 12h and level lag of 12h
  - results:
    - Average Train F1 Score: 0.9915501295944331
    - Average Train Precision: 0.9920147267242493
    - Average Test F1 Score: 0.10218923072315689
    - Average Test Precision: 0.14244917998092851

2. test 2 with 8 new cols
  - added redispatch and level lag of 12h, 24h, 36h, and 48h
  - results:
    - Average Train F1 Score: 0.9948075541756023
    - Average Train Precision: 0.9962690339292111
    - Average Test F1 Score: 0.08688445428317475
    - Average Test Precision: 0.13062294848350328

3. test 3 with 4 new cols
  - added redispatch and level lag of 12h, 24h
  - results:
    - Average Train F1 Score: 0.9936256724546532
    - Average Train Precision: 0.9949062498419048
    - Average Test F1 Score: 0.09647009249515685
    - Average Test Precision: 0.1450875350687569

4. test 4 with 4 new cols
  - added redispatch and level lag of 12h and 24h
  - XGBClassifier(max_depth=3, min_child_weight=5, gamma=0.1, subsample=0.8, colsample_bytree=0.8, eta=0.3, booster='gbtree', reg_alpha=4, reg_lambda=4, eval_metric='logloss', n_estimators=300, learning_rate=0.1, objective='binary:logistic', random_state = 42)
  - results:
    - Average Train F1 Score: 0.9688628383626243
    - Average Train Precision: 0.9677430890306062
    - Average Test F1 Score: 0.1571131836855964
    - Average Test Precision: 0.16138542398417716

5. test 5 with 4 new cols
  - added redispatch and level lag of 12h and 24h
  - XGBClassifier(max_depth=3, min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8, eta=0.3, booster='gbtree', reg_alpha=4, reg_lambda=4, eval_metric='logloss', n_estimators=300, learning_rate=0.1, objective='binary:logistic', random_state = 42)
  - Average Train F1 Score: 0.9693269618839041
  - Average Train Precision: 0.9682704611850873
  - Average Test F1 Score: 0.15884165476708328
  - Average Test Precision: 0.1635677273311264

6. test 6 with 4 new cols
  - added redispatch and level lag of 12h and 24h
  - XGBClassifier(max_depth=3, min_child_weight=1, gamma=0.1, subsample=0.6, colsample_bytree=0.8, eta=0.3, booster='gbtree', reg_alpha=4, reg_lambda=4, eval_metric='logloss', n_estimators=300, learning_rate=0.1, objective='binary:logistic', random_state = 42)
  - Average Train F1 Score: 0.9692256171812323
  - Average Train Precision: 0.9682865619100001
  - Average Test F1 Score: 0.15976504869085842
  - Average Test Precision: 0.16361172557789921

7. test 7 with 4 new cols
  - added redispatch and level lag of 12h and 24h
  - XGBClassifier(max_depth=3, min_child_weight=1, gamma=0.1, subsample=0.6, colsample_bytree=0.8, eta=0.3, booster='gbtree', reg_alpha=4, reg_lambda=4, eval_metric='logloss', n_estimators=300, learning_rate=0.1, objective='binary:logistic', random_state = 42)

Average Train F1 Score: 0.9691106998601267
Average Train Precision: 0.9680982007749634
Average Test F1 Score: 0.16098225252138793
Average Test Precision: 0.16372665012292364

8. Average Train F1 Score: 0.968794783239666
Average Train Precision: 0.9674467229696168
Average Test F1 Score: 0.1630161847366968
Average Test Precision: 0.1644600450967673

9. XGBClassifier(max_depth=3, min_child_weight=10, gamma=0.2, subsample=0.6, colsample_bytree=0.8, eta=0.3, booster='gbtree', reg_alpha=4, reg_lambda=4,
                          n_estimators=300, learning_rate=0.1, objective='binary:logistic', random_state=42)
    model.fit(X_train_resampled, y_train_resampled,
              eval_set=[(X_train_resampled, y_train_resampled), (X_test, y_test)],
              eval_metric='logloss',  Use the same eval_metric as specified in the model constructor
              early_stopping_rounds=early_stopping_rounds, verbose=False)

Average Train F1 Score: 0.9384962863576518
Average Train Precision: 0.9465448373438738
Average Test F1 Score: 0.21852579635509956
Average Test Precision: 0.18008337411938

10. model = XGBClassifier(max_depth=3, min_child_weight=10, gamma=0.2, subsample=0.5, colsample_bytree=0.5, eta=0.5,
                          booster='gbtree', reg_alpha=4, reg_lambda=4,
                          n_estimators=300, learning_rate=0.1, objective='binary:logistic', random_state=42, verbosity=0)

Average Train F1 Score: 0.9424078284610673
Average Train Precision: 0.9481400026555029
Average Test F1 Score: 0.22222587400809907
Average Test Precision: 0.18362724806775338

11. params = {
    'max_depth': 3,
    'min_child_weight': 10,
    'gamma': 0.2,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'eta': 0.5,
    'booster': 'gbtree',
    'reg_alpha': 4,
    'reg_lambda': 4,
    'n_estimators': 500,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'random_state': 42,
    'verbosity': 0,
    'early_stopping_rounds': 10  # Set early stopping rounds
}

Average Train F1 Score: 0.9446139030247693
Average Train Precision: 0.950068119071387
Average Test F1 Score: 0.22060981498923868
Average Test Precision: 0.18575056299446638

In [ ]:
df_lagged.columns

Index(['redispatch', 'level', 'wind_speed_m/s', 'wind_speed_m/s_lag1',
       'wind_speed_m/s_lag2', 'wind_speed_m/s_lag3', 'wind_speed_m/s_lag4',
       'wind_speed_m/s_lag5', 'wind_direction_degrees',
       'wind_direction_degrees_lag1', 'wind_direction_degrees_lag2',
       'wind_direction_degrees_lag3', 'wind_direction_degrees_lag4',
       'wind_direction_degrees_lag5', 'radiation_global_J/m2',
       'radiation_global_J/m2_lag1', 'radiation_global_J/m2_lag2',
       'radiation_global_J/m2_lag3', 'radiation_global_J/m2_lag4',
       'radiation_global_J/m2_lag5', 'air_temperature_K',
       'air_temperature_K_lag1', 'air_temperature_K_lag2',
       'air_temperature_K_lag3', 'air_temperature_K_lag4',
       'air_temperature_K_lag5', 'humidity_percent', 'humidity_percent_lag1',
       'humidity_percent_lag2', 'humidity_percent_lag3',
       'humidity_percent_lag4', 'humidity_percent_lag5', 'wind_gust_max_m/s',
       'wind_gust_max_m/s_lag1', 'wind_gust_max_m/s_lag2',
       'wind_g

In [ ]:
# Define features and target
features = ['wind_speed_m/s', 'wind_speed_m/s_lag1',
       'wind_direction_degrees', 'wind_direction_degrees_lag1',
       'radiation_global_J/m2', 'radiation_global_J/m2_lag1',
       'air_temperature_K', 'air_temperature_K_lag1', 'humidity_percent',
       'humidity_percent_lag1', 'wind_gust_max_m/s', 'wind_gust_max_m/s_lag1',
       'wind_direction_gust_max_degrees',
       'wind_direction_gust_max_degrees_lag1', 'forecast_solar_MW',
       'forecast_solar_MW_lag1', 'total_grid_load_MWh',
       'total_grid_load_MWh_lag1', 'residual_load_MWh',
       'residual_load_MWh_lag1', 'pumped_storage_MWh',
       'pumped_storage_MWh_lag1', 'redispatch_lag_48', 'level_lag_48',
       'redispatch_lag_96', 'level_lag_96']

target = 'redispatch'

X = df_lagged[features]
y = df_lagged[target]

# Define hyperparameters
params = {
    'max_depth': 3,
    'min_child_weight': 10,
    'gamma': 0.2,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'booster': 'gbtree',
    'reg_alpha': 4,
    'reg_lambda': 4,
    'n_estimators': 500,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'random_state': 42,
    'verbosity': 0,
    'early_stopping_rounds': 10,  # Set early stopping rounds
    'scale_pos_weight': 1
}

# Define the number of splits for time series cross-validation
n_splits = 10  # You can adjust this value as needed
gap = 48  # 12 hour difference between train and test sets

# Initialize lists to store evaluation metrics for each fold
train_f1_scores = []
train_precision_scores = []
test_f1_scores = []
test_precision_scores = []

# Define the time series splitter
tscv = TimeSeriesSplit(n_splits=n_splits, gap=gap)

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
    print(f"Training on fold {fold}/{n_splits}")
    # Get the data for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Handle class imbalance with SMOTE
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Initialize and train the XGBoost classifier with early stopping
    model = XGBClassifier(**params)

    eval_set = [(X_train_resampled, y_train_resampled), (X_test, y_test)]  # Evaluation sets for early stopping
    model.fit(X_train_resampled, y_train_resampled, eval_set=eval_set, verbose=0)

    # Predictions
    y_train_pred = model.predict(X_train_resampled)
    y_test_pred = model.predict(X_test)

    # Evaluate the model using precision and F1 score
    train_f1 = f1_score(y_train_resampled, y_train_pred)
    train_precision = precision_score(y_train_resampled, y_train_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)

    # Store the evaluation metrics for this fold
    train_f1_scores.append(train_f1)
    train_precision_scores.append(train_precision)
    test_f1_scores.append(test_f1)
    test_precision_scores.append(test_precision)

# Calculate average evaluation metrics across all folds
avg_train_f1 = np.mean(train_f1_scores)
avg_train_precision = np.mean(train_precision_scores)
avg_test_f1 = np.mean(test_f1_scores)
avg_test_precision = np.mean(test_precision_scores)

# Print the results
print("Average Train F1 Score:", avg_train_f1)
print("Average Train Precision:", avg_train_precision)
print("Average Test F1 Score:", avg_test_f1)
print("Average Test Precision:", avg_test_precision)

Training on fold 1/10
Training on fold 2/10
Training on fold 3/10
Training on fold 4/10
Training on fold 5/10
Training on fold 6/10
Training on fold 7/10
Training on fold 8/10
Training on fold 9/10
Training on fold 10/10
Average Train F1 Score: 0.9375259803395558
Average Train Precision: 0.9487513956232434
Average Test F1 Score: 0.2117713012567693
Average Test Precision: 0.17394025359851048


In [ ]:
y.value_counts()

0.0    131714
1.0      8398
Name: redispatch, dtype: int64

In [ ]:
df_lagged.columns

Index(['redispatch', 'level', 'wind_speed_m/s', 'wind_speed_m/s_lag1',
       'wind_speed_m/s_lag2', 'wind_speed_m/s_lag3', 'wind_speed_m/s_lag4',
       'wind_speed_m/s_lag5', 'wind_direction_degrees',
       'wind_direction_degrees_lag1', 'wind_direction_degrees_lag2',
       'wind_direction_degrees_lag3', 'wind_direction_degrees_lag4',
       'wind_direction_degrees_lag5', 'radiation_global_J/m2',
       'radiation_global_J/m2_lag1', 'radiation_global_J/m2_lag2',
       'radiation_global_J/m2_lag3', 'radiation_global_J/m2_lag4',
       'radiation_global_J/m2_lag5', 'air_temperature_K',
       'air_temperature_K_lag1', 'air_temperature_K_lag2',
       'air_temperature_K_lag3', 'air_temperature_K_lag4',
       'air_temperature_K_lag5', 'humidity_percent', 'humidity_percent_lag1',
       'humidity_percent_lag2', 'humidity_percent_lag3',
       'humidity_percent_lag4', 'humidity_percent_lag5', 'wind_gust_max_m/s',
       'wind_gust_max_m/s_lag1', 'wind_gust_max_m/s_lag2',
       'wind_g